<a href="https://colab.research.google.com/github/gokulprazath/SupportTicket_Agent/blob/main/SupportTicket_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Build a system specialized agents that analyze support tickets and determine
appropriate routing.

By Gokul prazath Sri.

## Notebook Overview

This notebook demonstrates how to build a support agent using `pydantic-ai` that interacts with a SQLite database to manage support tickets. The key steps covered are:

1.  **Installation**: Installing the necessary libraries, including `pydantic-ai`, `crewai`, `openai`, and `pydantic-ai-slim[groq]`.
2.  **Model Definition**: Defining the language model to be used by the agent. In this case, a Groq model is configured.
3.  **Agent Definition and Run**: Defining the `SupportAgent` with its dependencies, output type, and system prompt. This section also shows how to run the agent with a specific prompt and dependencies.
4.  **Database Operations (Manual)**:
    *   Creating a SQLite database and a `tickets` table.
    *   Inserting sample ticket data into the table.
    *   Displaying the data currently in the table.
    *   Removing all data from the table (optional).
5.  **Agent Tools**: Defining the tools that the `SupportAgent` can use to interact with the database:
    *   `insert_ticket`: To add new tickets.
    *   `extract_details`: To retrieve details of existing tickets.
    *   `update_details`: To modify details of existing tickets.

In [ ]:
!pip install pydantic-ai crewai openai pydantic-ai-slim[groq]

In [ ]:
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext2
from pydantic_ai.providers.google import GoogleProvider
from enum import Enum
from typing import Dict, Any, List, Optional
import sqlite3
from google.colab import userdata
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from openai import OpenAI

# Insert, display, remove the data in db

In [ ]:
def create_table(conn):
    with conn:
        conn.execute('''CREATE TABLE IF NOT EXISTS tickets (
                            ticket_id TEXT PRIMARY KEY NOT NULL,
                            customer_tier TEXT,
                            subject TEXT,
                            message TEXT,
                            previous_tickets INTEGER,
                            monthly_revenue REAL,
                            account_age_days INTEGER,
                            status TEXT,
                            priority INTEGER
                        )''')
def manual_insert_ticket(tick,db:Dict[str, Any])->str:
    conn = sqlite3.connect("/content/sample_data/ticket.db")
    cursor = conn.cursor()
    create_table(conn)
    # cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name=tickets;")
    # if cursor.fetchone() is not None:
    #     create_table(conn)

    chunk_size = 1000 # Define a chunk size for bulk inserts
    with conn:
        ticket_id = tick
        values = (
                ticket_id,
                db.get("customer_tier"),
                db.get("subject"),
                db.get("message"),
                db.get("previous_tickets"),
                db.get("monthly_revenue"),
                db.get("account_age_days"),
                "open"
        )
        print(values)

        placeholders = ", ".join(["(?, ?, ?, ?, ?, ?, ?,?)"] )
        sql = f"INSERT INTO tickets (ticket_id, customer_tier, subject, message, previous_tickets, monthly_revenue, account_age_days,status) VALUES {placeholders}"
        conn.execute(sql, values) # Corrected: Pass the tuple directly
    conn.close
    return "insertion successful"
manual_ticket_data = [
    {
        "ticket_id": "MAN-001",
        "customer_tier": "basic",
        "subject": "Login issue on mobile",
        "message": "Unable to log in using the mobile application.",
        "previous_tickets": 5,
        "monthly_revenue": 150,
        "account_age_days": 150,
        "status": "open"
    },
    {
        "ticket_id": "MAN-002",
        "customer_tier": "premium",
        "subject": "Feature request: Export data",
        "message": "Would be great to have an option to export data in CSV format.",
        "previous_tickets": 1,
        "monthly_revenue": 500,
        "account_age_days": 300,
        "status": "in-progress"
    },
    {
        "ticket_id": "MAN-003",
        "customer_tier": "free",
        "subject": "Question about pricing",
        "message": "I have a question about the different pricing tiers.",
        "previous_tickets": 0,
        "monthly_revenue": 0,
        "account_age_days": 10,

        "status": "closed"
    }
]
for i in manual_ticket_data:
    print(i)
    tick=i.get("ticket_id")
    insert_result =  manual_insert_ticket(tick,i)

{'ticket_id': 'MAN-001', 'customer_tier': 'basic', 'subject': 'Login issue on mobile', 'message': 'Unable to log in using the mobile application.', 'previous_tickets': 5, 'monthly_revenue': 150, 'account_age_days': 150, 'status': 'open'}
('MAN-001', 'basic', 'Login issue on mobile', 'Unable to log in using the mobile application.', 5, 150, 150, 'open')
{'ticket_id': 'MAN-002', 'customer_tier': 'premium', 'subject': 'Feature request: Export data', 'message': 'Would be great to have an option to export data in CSV format.', 'previous_tickets': 1, 'monthly_revenue': 500, 'account_age_days': 300, 'status': 'in-progress'}
('MAN-002', 'premium', 'Feature request: Export data', 'Would be great to have an option to export data in CSV format.', 1, 500, 300, 'open')
{'ticket_id': 'MAN-003', 'customer_tier': 'free', 'subject': 'Question about pricing', 'message': 'I have a question about the different pricing tiers.', 'previous_tickets': 0, 'monthly_revenue': 0, 'account_age_days': 10, 'status': 

In [ ]:
# prompt: display all data in db

conn = sqlite3.connect("/content/sample_data/ticket.db")
cursor = conn.cursor()

# Check if the table exists
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='tickets';")
if cursor.fetchone() is None:
    print("Table 'tickets' does not exist.")
else:
    cursor.execute("SELECT * FROM tickets")
    rows = cursor.fetchall()

    if not rows:
        print("The 'tickets' table is empty.")
    else:
        # Get column names
        column_names = [description[0] for description in cursor.description]

        # Print header
        print(column_names)
        # Print data rows
        for row in rows:
            print(row)

conn.close()


['ticket_id', 'customer_tier', 'subject', 'message', 'previous_tickets', 'monthly_revenue', 'account_age_days', 'status', 'priority']
('MAN-001', 'basic', 'Login issue on mobile', 'Unable to log in using the mobile application.', 5, 150.0, 150, 'open', None)
('MAN-002', 'premium', 'Feature request: Export data', 'Would be great to have an option to export data in CSV format.', 1, 500.0, 300, 'open', None)
('MAN-003', 'free', 'Question about pricing', 'I have a question about the different pricing tiers.', 0, 0.0, 10, 'open', None)


In [ ]:
# Run ONLY if u need to remove the data from db

conn = sqlite3.connect("/content/sample_data/ticket.db")
cursor = conn.cursor()

try:
    # Check if the table exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='tickets';")
    if cursor.fetchone() is None:
        print("Table 'tickets' does not exist.")
    else:
        # Delete all rows from the tickets table
        cursor.execute("DELETE FROM tickets")
        conn.commit()
        print("All data removed from the 'tickets' table.")

except sqlite3.Error as e:
    print(f"Error removing data: {e}")
finally:
    conn.close()

All data removed from the 'tickets' table.


# Agent definition

In [ ]:
import os
from pydantic_ai.models.groq import GroqModel
from pydantic_ai.providers.groq import GroqProvider

os.environ["GROQ_API_KEY"]="your_groq_api_key"# You can intialize in env or manually
model = GroqModel(
    'llama-3.3-70b-versatile',
    provider=GroqProvider(api_key='your_groq_api_key'),
)
from httpx import AsyncClient

from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.together import TogetherProvider

# model = OpenAIModel(
#     'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',  # model library available at https://www.together.ai/models
#     provider=TogetherProvider(api_key='your_key'),
# )


In [ ]:
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import Agent as PydanticAgent, RunContext
from pydantic_ai.providers.google import GoogleProvider
from enum import Enum
from typing import Dict, Any, List, Optional
import sqlite3
from google.colab import userdata
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from openai import OpenAI

@dataclass
class SupportDependencies:
    ticket_id: str

class Status(Enum):
    OPEN = "open"
    IN_PROGRESS = "in_PROGRESS"
    CLOSED = "closed"

class RiskLevel(Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"

class SupportOutput(BaseModel): # Useful for getting structured output.
    support_advice: str = Field(description='Advice returned to the customer')
    ticket_priority: Optional[int] = Field(description="Priority given to the ticket based on the ticket details", ge=0, le=5)
    status: Optional[Status] = Field(description='Status of the ticket')
    ticket_details:  Optional[Dict[str, Any]] = Field(description='Details about the ticket as a list of JSON object which can be extracted with the help of tools')

support_agent = PydanticAgent(
    model=model, # Pass the model as a keyword argument
    deps_type=SupportDependencies,
    system_prompt=(
   """You are a support agent in our company, responsible for assisting customers with managing support tickets.\n\n
        Your tasks include:\n
        - Creating new tickets using the `insert_ticket` function. Input: db: Dict[str, Any]. Each entry in the list is a row (JSON) representing ticket data.\n
        - Extracting details of one or more tickets using the `extract_details` function. Input: columns: List[str], which specifies which fields to extract for the given ticket_id(s).\n
        - Updating one or more tickets using the `update_details` function.\n\n
        Your main responsibilities are:\n
        - Support users in creating new tickets by collecting and inserting ticket_details.\n
        - Assist users in updating specific fields in a ticket using `update_details` and displaying the updated result.\n
        - Automatically generate the priority of a ticket based on the inserted details and update the record accordingly.\n
        - Ensure clear and helpful communication with users throughout the support process.\n\n
        You can use multilple tools according to the input request. Always ensure accuracy and integrity of ticket data. Confirm updates or extractions with clear, concise responses."""
    ),
)

# Tool Definition

In [ ]:

@support_agent.tool
async def insert_ticket(deps:RunContext[SupportDependencies],db:Dict[str, Any]):
    """Inserts a new ticket into the database."""
    print("Insertion started")
    conn = sqlite3.connect("/content/sample_data/ticket.db")
    cursor = conn.cursor()
    # cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name=tickets;")
    # if cursor.fetchone() is not None:
    #     create_table(conn)

    with conn:
        ticket_id = deps.deps.ticket_id
        values = [
                ticket_id,
                db.get("customer_tier"),
                db.get("subject"),
                db.get("message"),
                db.get("previous_tickets"),
                db.get("monthly_revenue"),
                db.get("account_age_days"),
                "open"
        ]
        placeholders = ", ".join(["(?, ?, ?, ?, ?, ?, ?,?)"] )
        sql = f"INSERT INTO tickets (ticket_id, customer_tier, subject, message, previous_tickets, monthly_revenue, account_age_days,status) VALUES {placeholders}"
        conn.execute(sql, values)
        print("Insertion successfully completed details",db)
    conn.close()
    return "insertion successful"

In [ ]:

@support_agent.tool
async def update_details(ticket_id:RunContext[SupportDependencies],ticket_updates:Dict[str,Any]) -> str:
    """Updates details for a given ticket ID in the database."""
    print("Update started")
    conn = sqlite3.connect("/content/sample_data/ticket.db")
    cursor = conn.cursor()
    print(ticket_id.deps.ticket_id)
    updated_count = 0
    with conn:
        ticket_id = ticket_id.deps.ticket_id

        # Build the SET clause for the UPDATE query
        set_clauses = []
        values = []
        for key, value in ticket_updates.items():
            if key != "ticket_id":
                set_clauses.append(f"{key} = ?")
                values.append(value)

        # Add ticket_id to values for the WHERE clause
        values.append(ticket_id)

        if not set_clauses: # Check if there are any fields to update
            conn.close()
            return f"No updatable fields provided for ticket {ticket_id}."

        sql_query = f"UPDATE tickets SET {', '.join(set_clauses)} WHERE ticket_id = ?"

        try:
            cursor.execute(sql_query, values)
            print(f"Update completed details for {ticket_id}",ticket_updates)
        except sqlite3.Error as e:
            print(f"Error updating ticket {ticket_id}: {e}")


    conn.close()
    return f"Successfully attempted to update details for ticket {ticket_id}"

In [ ]:
@support_agent.tool
async def extract_details(ticket_id:RunContext[SupportDependencies],columns:List[str]) -> Dict[str, Any]:
    """Extracts details for a given ticket ID from the database."""
    print("Extraction started")
    conn = sqlite3.connect("/content/sample_data/ticket.db")
    cursor = conn.cursor()
    ticket_id= ticket_id.deps.ticket_id
    print(ticket_id,columns)
    if 'ticket_id' not in columns:
        columns.append('ticket_id')



    # Create the SELECT query
    columns_str = ", ".join(columns)
    sql_query = f"SELECT {columns_str} FROM tickets WHERE ticket_id = ?"
    cursor.execute(sql_query, (ticket_id,)) # Pass ticket_id as a tuple
    rows = cursor.fetchall()
    row=rows[0]
    result = {}
    for i, col in enumerate(columns):
        result[col] = row[i]
    conn.close()
    print("Extraction completed and the output is ",result)
    return result,"Extraction Successfull and the details are "

# Agent run

In [ ]:
insert_prompt="""Call the `insert_ticket` tool with the following ticket details as the `db` argument:
{"ticket_id": "assfd1234",
    "customer_tier": "enterprise",
"subject": "Urgent: Security vulnerability?",
"message": "Our security team flagged that your API responses include internal server paths in",
"previous_tickets": 20,
"monthly_revenue": 50000,
"account_age_days": 900}
"""
extract_prompt="Extract the customer_tier,subject,monthly_revenue related to ticket_id assfd123 using extract_details tool"
update_prompt="Update the monthly_revenue for the ticket assfd123 to 91999"

In [ ]:

deps = SupportDependencies(ticket_id="asssfd1234")
result = await support_agent.run(insert_prompt ,deps=deps)
print(result.output)

Insertion started
Insertion successfully completed details {'account_age_days': 900, 'customer_tier': 'enterprise', 'message': 'Our security team flagged that your API responses include internal server paths in', 'monthly_revenue': 50000, 'previous_tickets': 20, 'subject': 'Urgent: Security vulnerability?', 'ticket_id': 'assfd1234'}
Update started
asssfd1234
Update completed details for asssfd1234 {'priority': 'high', 'ticket_id': 'assfd1234'}
The ticket has been inserted and updated with a high priority.


In [ ]:
deps = SupportDependencies(ticket_id="assfd1234")
result = await support_agent.run(extract_prompt ,deps=deps)
print(result.output)

Extraction started
assfd1234 ['customer_tier', 'subject', 'monthly_revenue']
Extraction completed and the output is  {'customer_tier': 'enterprise', 'subject': 'Urgent: Security vulnerability?', 'monthly_revenue': 50000.0, 'ticket_id': 'assfd1234'}


In [ ]:
deps = SupportDependencies(ticket_id="assfd1234")
result = await support_agent.run(update_prompt ,deps=deps)
print(result.output)

Update started
assfd1234
Update completed details for assfd1234 {'monthly_revenue': 91999, 'ticket_id': 'assfd123'}
Extraction started
assfd1234 ['monthly_revenue']
Extraction completed and the output is  {'monthly_revenue': 91999.0, 'ticket_id': 'assfd1234'}
The monthly_revenue for the ticket assfd123 has been successfully updated to 91999. The updated details are: monthly_revenue: 91999.


In [ ]:
print(result.all_messages())

[ModelRequest(parts=[SystemPromptPart(content='You are a support agent in our company, responsible for assisting customers with managing support tickets.\n\n\n        Your tasks include:\n\n        - Creating new tickets using the `insert_ticket` function. Input: db: Dict[str, Any]. Each entry in the list is a row (JSON) representing ticket data.\n\n        - Extracting details of one or more tickets using the `extract_details` function. Input: columns: List[str], which specifies which fields to extract for the given ticket_id(s).\n\n        - Updating one or more tickets using the `update_details` function.\n\n\n        Your main responsibilities are:\n\n        - Support users in creating new tickets by collecting and inserting ticket_details.\n\n        - Assist users in updating specific fields in a ticket using `update_details` and displaying the updated result.\n\n        - Automatically generate the priority of a ticket based on the inserted details and update the record accordi

In [ ]:
print("""[ModelRequest(parts=[SystemPromptPart(content='You are a support agent in our company, responsible for assisting customers with managing support tickets.\n\n\n        Your tasks include:\n\n        - Creating new tickets using the `insert_ticket` function. Input: db: Dict[str, Any]. Each entry in the list is a row (JSON) representing ticket data.\n\n        - Extracting details of one or more tickets using the `extract_details` function. Input: columns: List[str], which specifies which fields to extract for the given ticket_id(s).\n\n        - Updating one or more tickets using the `update_details` function.\n\n\n        Your main responsibilities are:\n\n        - Support users in creating new tickets by collecting and inserting ticket_details.\n\n        - Assist users in updating specific fields in a ticket using `update_details` and displaying the updated result.\n\n        - Automatically generate the priority of a ticket based on the inserted details and update the record accordingly.\n\n        - Ensure clear and helpful communication with users throughout the support process.\n\n\n        Always ensure accuracy and integrity of ticket data. Confirm updates or extractions with clear, concise responses.', timestamp=datetime.datetime(2025, 6, 23, 4, 2, 11, 621778, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Call the `insert_ticket` tool with the following ticket details as the `db` argument:\n{"customer_tier": "enterprise",\n"subject": "Urgent: Security vulnerability?",\n"message": "Our security team flagged that your API responses include internal server paths in",\n"previous_tickets": 20,\n"monthly_revenue": 50000,\n"account_age_days": 900}\n', timestamp=datetime.datetime(2025, 6, 23, 4, 2, 11, 621789, tzinfo=datetime.timezone.utc))]), ModelResponse(parts=[ToolCallPart(tool_name='insert_ticket', args='{"db":{"account_age_days":900,"customer_tier":"enterprise","message":"Our security team flagged that your API responses include internal server paths in","monthly_revenue":50000,"previous_tickets":20,"subject":"Urgent: Security vulnerability?"}}', tool_call_id='call_0465i2zvpdi38zqvwf78t80q')], usage=Usage(requests=1, request_tokens=967, response_tokens=81, total_tokens=1048, details={}), model_name='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', timestamp=datetime.datetime(2025, 6, 23, 4, 2, 11, tzinfo=TzInfo(UTC)), vendor_id='nzE4g9Y-4Yz4kd-954116a708e82250'), ModelRequest(parts=[ToolReturnPart(tool_name='insert_ticket', content='insertion successful', tool_call_id='call_0465i2zvpdi38zqvwf78t80q', timestamp=datetime.datetime(2025, 6, 23, 4, 2, 13, 204246, tzinfo=datetime.timezone.utc))]), ModelResponse(parts=[TextPart(content='The function call to `insert_ticket` has been executed with the provided ticket details as the `db` argument. The ticket details include the customer tier, subject, message, previous tickets, monthly revenue, and account age in days.')], usage=Usage(requests=1, request_tokens=1048, response_tokens=48, total_tokens=1096, details={}), model_name='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', timestamp=datetime.datetime(2025, 6, 23, 4, 2, 13, tzinfo=TzInfo(UTC)), vendor_id='nzE4gZe-3NKUce-954116b098952250'), ModelRequest(parts=[RetryPromptPart(content='Plain text responses are not permitted, please include your response in a tool call', tool_call_id='pyd_ai_8399268baa5b43748519c0ecc9ec7e2b', timestamp=datetime.datetime(2025, 6, 23, 4, 2, 16, 940763, tzinfo=datetime.timezone.utc))]), ModelResponse(parts=[ToolCallPart(tool_name='final_result', args='{"status":"open","support_advice":"The function call to insert_ticket has been executed with the provided ticket details as the db argument. The ticket details include the customer tier, subject, message, previous tickets, monthly revenue, and account age in days.","ticket_details":{"account_age_days":900,"customer_tier":"enterprise","message":"Our security team flagged that your API responses include internal server paths in","monthly_revenue":50000,"previous_tickets":20,"subject":"Urgent: Security vulnerability?"},"ticket_priority":3}', tool_call_id='call_iyprc29d1qw98afu5udhnvw5')], usage=Usage(requests=1, request_tokens=1128, response_tokens=147, total_tokens=1275, details={}), model_name='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', timestamp=datetime.datetime(2025, 6, 23, 4, 2, 23, tzinfo=TzInfo(UTC)), vendor_id='nzE4jkL-3NKUce-954116f38883f83e'), ModelRequest(parts=[ToolReturnPart(tool_name='final_result', content='Final result processed.', tool_call_id='call_iyprc29d1qw98afu5udhnvw5', timestamp=datetime.datetime(2025, 6, 23, 4, 2, 25, 245247, tzinfo=datetime.timezone.utc))])]

""")

[ModelRequest(parts=[SystemPromptPart(content='You are a support agent in our company, responsible for assisting customers with managing support tickets.


        Your tasks include:

        - Creating new tickets using the `insert_ticket` function. Input: db: Dict[str, Any]. Each entry in the list is a row (JSON) representing ticket data.

        - Extracting details of one or more tickets using the `extract_details` function. Input: columns: List[str], which specifies which fields to extract for the given ticket_id(s).

        - Updating one or more tickets using the `update_details` function.


        Your main responsibilities are:

        - Support users in creating new tickets by collecting and inserting ticket_details.

        - Assist users in updating specific fields in a ticket using `update_details` and displaying the updated result.

        - Automatically generate the priority of a ticket based on the inserted details and update the record accordingly.

        - E

# Challenges Faced



*   Each model in pydantic_ai has its own features, advantages and disadvantages.

    1.   Gpt-4o--> very good model and can understand the simple prompt and do the needed change. But not available freely needs to pay.     
    2.   Gemini--> Single api call works good and provides detailed output. Doesn't support tool calls completely. But available in free tier.
    3.   Groq--> Works pretty good with proper prompting. Limited usage which makes difficult to play with it.
    4.   Crew AI--> More volatile and needs different kind of definition when compared to all of these.
    5.   Together AI--> It just supports needs more improvement.


*   Pydantic Agents are weaker than autogen agent while comparing with tool usage. In autogen tools can be called easily with the customized input and description is more than enough but here dependecies restricts the llm to understand certain things. For these usecase understanding of data is not needed much but for complex problems I would prefer autogen, Lang graph.




# What can be done



*   Ticketing system can be build for categorizing, planning, automating the process.
    1. Once the user files a ticket provide a documentation steps related to the problem.
    2. If still facing issue then we should priorotize and categorize the issue like hardware issue, software issue, etc.. And can assign priority.
    3. Assign the ticket to respective department and assign it based on difficulty eg: the easy tickets to juniors complex one to seniors.

*   MCP, RAG can be utilized for efficient usage and integration. And output will be accurate.



> ✨ Thanks for viewing, it just takes 10 mins to run :)

